In [1]:
import re
import json
import argparse
import copy



In [2]:
parser = argparse.ArgumentParser(prog='inducelinearvariable', prefix_chars='-', description='induce linear variables',
                                 epilog="before using, check the input file")

parser.add_argument('filename', help='the filename of input json', type=str, nargs=1)

cond_f = {"==": lambda x, y: x == y, "!=": lambda x, y: x != y, ">=": lambda x, y: x >=
          y, "<=": lambda x, y: x <= y, ">": lambda x, y: x > y, "<": lambda x, y: x < y}
arith_f = {"+": lambda x, y: x+y, "-": lambda x, y: x-y, "*": lambda x,
           y: x*y, "/": lambda x, y: x//y, "%": lambda x, y: x % y}
eserved_word = {"HALT", "=", "+", "-", "*", "/", "%", "?",
                ":", "!:", ">", "<", ">=", "<=", "==", "!=", "[", "]"}
reg = re.compile('^[0-9]+$')



In [3]:
testjson='''
{
  "summary": {
    "total_blocks": 4
  },
  "blocks": {
    "0": {
      "line_num": [
        0,
        2
      ],
      "next": [
        "1",
        null
      ],
      "code": [
        "S = 0",
        "I = 1",
        "N = 100"
      ],
      "defd": [
        "S",
        "I",
        "N"
      ],
      "used": [],
      "in": [],
      "out": [
        "N",
        "S",
        "I"
      ],
      "pre": [],
      "dom": [
        "0"
      ]
    },
    "1": {
      "line_num": [
        3,
        3
      ],
      "next": [
        "2",
        "3"
      ],
      "code": [
        "? I > N : 9"
      ],
      "defd": [],
      "used": [
        "N",
        "I"
      ],
      "in": [
        "N",
        "S",
        "I"
      ],
      "out": [
        "S",
        "I",
        "N"
      ],
      "pre": [
        "2",
        "0"
      ],
      "dom": [
        "0",
        "1"
      ]
    },
    "2": {
      "line_num": [
        4,
        8
      ],
      "next": [
        "1",
        null
      ],
      "remainedvar":[
          "2",
          "1"
      ],
      "code": [
        "J = I * 2",
        "J = J - 1",
        "S = S + J",
        "I = I + 2",
        "!: 3"
      ],
      "defd": [
        "J"
      ],
      "used": [
        "S",
        "I"
      ],
      "in": [
        "S",
        "N",
        "I"
      ],
      "out": [
        "N",
        "S",
        "I"
      ],
      "pre": [
        "1"
      ],
      "dom": [
        "2",
        "1",
        "0"
      ]
    },
    "3": {
      "line_num": [
        9,
        9
      ],
      "next": [
        null,
        null
      ],
      "code": [
        "HALT"
      ],
      "defd": [],
      "used": [],
      "in": [],
      "out": [],
      "pre": [
        "1"
      ],
      "dom": [
        "0",
        "3",
        "1"
      ]
    }
  },
  "loops": [
    {
      "loop_blks": [
        "1",
        "2"
      ],
      "back_edge": [
        "2",
        "1"
      ]
    }
  ]
}
'''

In [4]:
program=json.loads(testjson)

def appendblk(loopkey):
    #循环
    loop = program['loops'][loopkey]
    #回基本块
    backblk = program['blocks'][loop['back_edge'][1]]

    #新基本块
    maxblock = program['summary']['total_blocks']
    maxblock += 1
    program['blocks'][str(maxblock)] = {
        "line_num": [0, 0],
        "next": [loop['back_edge'][1],None],
        "code": [],
        "defd": [],
        "remainedvar": [],
        "used": [],
        "in": [],
        "out": [],
        "pre": list(set(backblk['pre'])-set(loop['back_edge'][0])),
        "dom": []
    }
    program['summary']['total_blocks'] = maxblock
    newblk=program['blocks'][str(maxblock)]
    
    #改变所有权
    for blkname in newblk['pre']:
        blk = program['blocks'][blkname]
        if(blk['next'][0] == loop['back_edge'][1]):
            blk['next'][0] = str(maxblock)
        if(blk['next'][1] == loop['back_edge'][1]):
            blk['next'][1] = str(maxblock)

    backblk['pre'] = [str(maxblock), loop['back_edge'][0]]
    
    loop["loop_blks"].append(str(maxblock))

    return newblk


In [5]:
# 处理头
for loop in program['loops']:
    head=[]
    for blk_name in loop['loop_blks']:
        blk=program['blocks'][blk_name]
        # 回边指向或唯一前驱
        if(blk_name == loop['back_edge'][1] or len(blk['pre'])<=1):
            head.append(blk_name)
        else:
            # 头必须连续
            break
    loop['head']=head

print(program['loops'])

[{'loop_blks': ['1', '2'], 'back_edge': ['2', '1'], 'head': ['1', '2']}]


In [6]:
def appear(var,code):
    if(re.match('^(.)*=(.)*$', code)):
        if(re.match(r'^[A-Za-z_][A-Za-z0-9_]*[(.)*]$', var)):
            return False
        thisvar = re.search('[A-Za-z_][A-Za-z0-9_]*', code).group(0)
        if(re.match(var,thisvar)):
            return True
    return False

def appearonlyonce(var,codes):
    i=0
    cur=0
    while(i<len(codes)):
        app=0
        while(appear(var,codes[i])):
            i+=1
            app=1
        cur+=app
        i+=1
    return cur==1

def appearsingleline(var,codes):
    cur=0
    for code in codes:
        if(appear(var,code)):
            cur+=1
    return cur==1

# 求唯一定值段

def getassigned(blk):
    codes=blk['code']
    # 求被定值量
    variables=[]
    for code in codes:
        # 是赋值表达式
        if(re.match('^(.)*=(.)*$',code)):
            var = re.search('[A-Za-z_][A-Za-z0-9_]*', code).group(0)
            # 数组赋值不考虑
            if(re.match(r'^[A-Za-z_][A-Za-z0-9_]*[(.)*]$',var)):
                pass
            else:
                # 定值段唯一
                if(appearonlyonce(var,codes)):
                    variables=list(set(variables)|set([var]))

    return variables


def judgeselfplus(var,code):
    return appearsingleline(var,code) 
    and (re.match(r"^(\s)*"+var+r"(\s)*=(\s)*"+var+r"(\s)*[\+\-](\s)*[0-9]+(\s)*$", code)
       or re.match(r"^(\s)*"+var+r"(\s)*=(\s)*[0-9]+(\s)*[\+\-](\s)*"+var+r"(\s)*$", code))


def judgestartinduce(var,codes,codeidx):

    # 判断可归纳

    return False
    

def dealwithbasic():
    pass

def dealwithinduce():
    pass


def dealwithvar(var, blk, newblk):
    print(var,"basic")
    codes = blk['code']
    #print(codes)
    for codeidx in range(len(codes)):
        code=codes[codeidx]
        print(code)
        # 是赋值表达式
        if(re.match('^(.)*=(.)*$', code)):
            thisvar = re.search('[A-Za-z_][A-Za-z0-9_]*', code).group(0)
            # 数组赋值不考虑
            if(re.match(r'^[A-Za-z_][A-Za-z0-9_]*[(.)*]$', thisvar)):
                pass
            else:
                # 定值段唯一
                if(var==thisvar):
                    #为基本变量
                    if(judgeselfplus(var,code)):
                       
                       # 处理基本变量
                       
                    else if(judgestartinduce(var,codes,codeidx)):

                        # 处理可归纳
                        
                    return False



for loopidx in range(len(program['loops'])):

    # 新建块
    loop = program['loops'][loopidx]
    #print(loopidx)
    newblk = appendblk(loopidx)
    
    # 对于每一个循环
    for blk_name in loop['head']:
        blk = program['blocks'][blk_name]
        blk['assigned']=getassigned(blk)
    
    print(loop['head'])

    for blk_name in loop['head']:
        print(blk_name,"name")
        blk = program['blocks'][blk_name]
        print(blk)
        for var in blk['assigned']:
            dealwithvar(var,blk,newblk)

    #print(newblk)


program['blocks']


['1', '2']
1 name
{'line_num': [3, 3], 'next': ['2', '3'], 'code': ['? I > N : 9'], 'defd': [], 'used': ['N', 'I'], 'in': ['N', 'S', 'I'], 'out': ['S', 'I', 'N'], 'pre': ['5', '2'], 'dom': ['0', '1'], 'assigned': []}
2 name
{'line_num': [4, 8], 'next': ['1', None], 'remainedvar': ['2', '1'], 'code': ['J = I * 2', 'J = J - 1', 'S = S + J', 'I = I + 2', '!: 3'], 'defd': ['J'], 'used': ['S', 'I'], 'in': ['S', 'N', 'I'], 'out': ['N', 'S', 'I'], 'pre': ['1'], 'dom': ['2', '1', '0'], 'assigned': ['S', 'I', 'J']}
S basic
J = I * 2
J = J - 1
S = S + J
I = I + 2
!: 3
I basic
J = I * 2
J = J - 1
S = S + J
I = I + 2
I
!: 3
J basic
J = I * 2
J = J - 1
J
S = S + J
I = I + 2
!: 3


{'0': {'line_num': [0, 2],
  'next': ['5', None],
  'code': ['S = 0', 'I = 1', 'N = 100'],
  'defd': ['S', 'I', 'N'],
  'used': [],
  'in': [],
  'out': ['N', 'S', 'I'],
  'pre': [],
  'dom': ['0']},
 '1': {'line_num': [3, 3],
  'next': ['2', '3'],
  'code': ['? I > N : 9'],
  'defd': [],
  'used': ['N', 'I'],
  'in': ['N', 'S', 'I'],
  'out': ['S', 'I', 'N'],
  'pre': ['5', '2'],
  'dom': ['0', '1'],
  'assigned': []},
 '2': {'line_num': [4, 8],
  'next': ['1', None],
  'remainedvar': ['2', '1'],
  'code': ['J = I * 2', 'J = J - 1', 'S = S + J', 'I = I + 2', '!: 3'],
  'defd': ['J'],
  'used': ['S', 'I'],
  'in': ['S', 'N', 'I'],
  'out': ['N', 'S', 'I'],
  'pre': ['1'],
  'dom': ['2', '1', '0'],
  'assigned': ['S', 'I', 'J']},
 '3': {'line_num': [9, 9],
  'next': [None, None],
  'code': ['HALT'],
  'defd': [],
  'used': [],
  'in': [],
  'out': [],
  'pre': ['1'],
  'dom': ['0', '3', '1']},
 '5': {'line_num': [0, 0],
  'next': ['1', None],
  'code': [],
  'defd': [],
  'remainedvar':

if __name__ == "__main__":
    # 读文件
    args = parser.parse_args()
    filename = args.filename[0]
    with open(filename, "r") as fp:
        vm_blk = json.load(fp)
    blocks = vm_blk["blocks"]
    # 计算基本块的前驱
    for n, b in blocks.items():
        blocks[n]["pre"] = set()
    for n, b in blocks.items():
        for i in b["next"]:
            if i != None:
                blocks[i]["pre"] = blocks[i]["pre"] | {n}
    # 构建支配关系
    for n, b in blocks.items():
        blocks[n]["dom"] = set(blocks.keys())
    blocks[str(0)]["dom"] = {'0'}
    flag = True
    while flag:
        flag = False
        for n, b in blocks.items():
            print(b["dom"])
        for n, b in blocks.items():
            if n != '0':
                domset_ori = copy.deepcopy(blocks[n]["dom"])
                domset = copy.deepcopy(blocks[n]["dom"])
                preset = set(blocks.keys())
                for j in blocks[n]["pre"]:
                    preset = preset & blocks[j]["dom"]
                domset = {n} | preset
                if (domset != domset_ori):
                    flag = True
                blocks[n]["dom"] = domset
    for n, b in blocks.items():
        if len(b["pre"]) == 0 and n != '0':
            blocks[n]["dom"] = set()
    # 寻找回边
    back_edge = []
    for n, b in blocks.items():
        for i in b["dom"]:
            if i in b["next"]:
                back_edge.append((n, i))
    back_edge = list(set(back_edge))
    # 寻找回边对应的自然循环
    loops = []
    for le in back_edge:
        stack = [le[0]]
        loop = {le[0], le[1]}
        if le[0] == le[1]:
            continue
        while len(stack) > 0:
            m = stack[-1]
            stack.pop()
            for p in blocks[m]["pre"]:
                if p not in loop:
                    loop = loop | {p}
                    stack.append(p)
        loop = [i for i in list(loop) if len(blocks[i]["pre"]) > 0]
        # print(le,sorted(loop))
        loops.append((sorted(loop), le))
    # 找只有一个基本块的循环
    for n, b in blocks.items():
        if n in b["next"]:
            loops.append(([n], (n, n)))
    loops = sorted(loops, key=lambda x: len(x[0]))
    # loops = list(set(loops))
    loops_res = []
    for l in loops:
        if l not in loops_res:
            loops_res.append({"loop_blks": l[0], "back_edge": l[1]})
    print(loops_res)
    for n, b in blocks.items():
        blocks[n]["dom"] = list(blocks[n]["dom"])
        blocks[n]["pre"] = list(blocks[n]["pre"])
    vm_blk["blocks"] = blocks
    vm_blk["loops"] = loops_res
    new_file = re.sub('(.*)\\.json$', r'\g<1>_loops.json', args.filename[0])
    print("Saving output to:", new_file)
    with open(new_file, "w") as fp:
        json.dump(vm_blk, fp, indent=2)
